In [ ]:

import numpy as np
import pandas as pd
import netCDF4 as nc
import geopandas as gpd
from netCDF4 import num2date
import datetime

from tqdm import tqdm

tqdm.pandas()
from shapely.geometry import Point

import matplotlib.pyplot as plt

import platform

from cartopy import crs as ccrs

if platform.system() == 'Darwin':
    plt.rcParams["font.family"] = 'Arial Unicode MS'
elif platform.system() == 'Windows':
    plt.rcParams["font.family"] = 'SimHei'
else:
    pass

In [ ]:

nc_data = nc.Dataset("cru_ts4.05.1901.2020.tmp.dat.nc")

## 提取变量


In [ ]:
raw_lat_data = np.array(nc_data.variables['lat'])
raw_lon_data = np.array(nc_data.variables['lon'])
raw_time_data = np.array(nc_data.variables['time'])
raw_tmp_data = np.array(nc_data.variables['tmp'])


In [ ]:
tmp_missing_value = nc_data.variables['tmp'].missing_value

## 导入中国边界数据

In [ ]:
china_boundary = gpd.read_file(filename="数据集/中国地图边界202111版.json")

china_boundary_valid = china_boundary.copy()
china_boundary_valid['geometry'] = china_boundary.buffer(0)


In [ ]:
def pic(lon, lat) -> bool:
    """
    检测一个点是否在中国边界线内
    lon:东经
    lat:北纬
    :param lon:
    :param lat:
    :return:
    """
    return china_boundary_valid.contains(Point(lon, lat))[0]

## 开始


In [ ]:
raw_lon_data,
raw_lat_data

In [ ]:
single_data = raw_tmp_data[1,:,:].copy()
single_data[single_data == tmp_missing_value] = np.nan

In [ ]:
fig, ax = plt.subplots()
ax.imshow(single_data[::-1, :])

## 裁切


In [ ]:
mask_matrix = np.full(shape=(360, 720), fill_value=False)
# mask_matrix[0,0] = True

In [ ]:
# single_data[mask_matrix] = 20
# single_data

In [ ]:
mask_matrix = np.full(shape=(360, 720), fill_value=False)
for temp_lat in tqdm(range(360)):
    for temp_lon in range(720):
        point_lat_lon = (raw_lon_data[temp_lon], raw_lat_data[temp_lat])
        mask_matrix[temp_lat, temp_lon] = pic(lon=point_lat_lon[0], lat=point_lat_lon[1])


In [ ]:
print('ok~')

In [ ]:
single_data[~mask_matrix] = np.nan

In [ ]:
fig, ax = plt.subplots()
ax.imshow(single_data[::-1, :])

In [ ]:
import rasterio
def array2gtiff(array, filename):
    """
    将一个矩阵保存为tiff文件,
    这里还可以设置tiff的crs和transofrm。更多，可以查看rasterio的官网或者下面的这个链接
    https://gis.stackexchange.com/questions/279953/numpy-array-to-gtiff-using-rasterio-without-source-raster
    :param array: shape:(row, col)
    :param filename:
    :return:
    """
    with rasterio.open(filename, 'w', driver='GTiff',
                       height=array.shape[0], width=array.shape[1],
                       count=1, dtype=str(array.dtype)) as f:
        f.write(array, 1)

# test function
array2gtiff(array=single_data[::-1,:], filename="结果/testchina.tiff")